# [1211. Queries Quality and Percentage](https://leetcode.com/problems/queries-quality-and-percentage/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Queries

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| query_name  | varchar |
| result      | varchar |
| position    | int     |
| rating      | int     |
+-------------+---------+</pre>
This table may have duplicate rows.
This table contains information collected from some queries on a database.
The position column has a value from 1 to 500.
The rating column has a value from 1 to 5. Query with rating less than 3 is a poor query.
 

We define query quality as:

The average of the ratio between query rating and its position.

We also define poor query percentage as:

The percentage of all queries with rating less than 3.

Write a solution to find each query_name, the quality and poor_query_percentage.

Both quality and poor_query_percentage should be rounded to 2 decimal places.

Return the result table in any order.

The result format is in the following example.

 

Example 1:

Input: 
Queries table:
<pre>+------------+-------------------+----------+--------+
| query_name | result            | position | rating |
+------------+-------------------+----------+--------+
| Dog        | Golden Retriever  | 1        | 5      |
| Dog        | German Shepherd   | 2        | 5      |
| Dog        | Mule              | 200      | 1      |
| Cat        | Shirazi           | 5        | 2      |
| Cat        | Siamese           | 3        | 3      |
| Cat        | Sphynx            | 7        | 4      |
+------------+-------------------+----------+--------+</pre>
Output: 
<pre>+------------+---------+-----------------------+
| query_name | quality | poor_query_percentage |
+------------+---------+-----------------------+
| Dog        | 2.50    | 33.33                 |
| Cat        | 0.66    | 33.33                 |
+------------+---------+-----------------------+</pre>
Explanation: 
Dog queries quality is ((5 / 1) + (5 / 2) + (1 / 200)) / 3 = 2.50
Dog queries poor_ query_percentage is (1 / 3) * 100 = 33.33

Cat queries quality equals ((2 / 5) + (3 / 3) + (4 / 7)) / 3 = 0.66
Cat queries poor_ query_percentage is (1 / 3) * 100 = 33.33

In [1]:
#pandas schema
import pandas as pd

data = [['Dog', 'Golden Retriever', 1, 5], ['Dog', 'German Shepherd', 2, 5], ['Dog', 'Mule', 200, 1],
        ['Cat', 'Shirazi', 5, 2], ['Cat', 'Siamese', 3, 3], ['Cat', 'Sphynx', 7, 4]]
queries = pd.DataFrame(data, columns=['query_name', 'result', 'position', 'rating']).astype(
    {'query_name': 'object', 'result': 'object', 'position': 'Int64', 'rating': 'Int64'})

In [2]:
#to pyspark df

from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

queries_df = spark.createDataFrame(queries)
queries_df.show(truncate=False)

bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
23/11/09 13:00:22 WARN Utils: Your hostname, svchost resolves to a loopback address: 127.0.1.1; using 172.18.28.34 instead (on interface eth0)
23/11/09 13:00:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/09 13:00:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+----------+----------------+--------+------+
|query_name|result          |position|rating|
+----------+----------------+--------+------+
|Dog       |Golden Retriever|1       |5     |
|Dog       |German Shepherd |2       |5     |
|Dog       |Mule            |200     |1     |
|Cat       |Shirazi         |5       |2     |
|Cat       |Siamese         |3       |3     |
|Cat       |Sphynx          |7       |4     |
+----------+----------------+--------+------+


In [3]:
# solving in spark dataframe

queries_df \
    .groupBy('query_name') \
    .agg(
    F.round(F.avg(F.col('rating') / F.col('position')), 2).alias('quality'),
    F.round(F.avg(F.when(F.col('rating') < 3, 1).otherwise(0)) * 100, 2).alias('poor_query_percentage')
).show()

+----------+-------+---------------------+
|query_name|quality|poor_query_percentage|
+----------+-------+---------------------+
|       Dog|    2.5|                33.33|
|       Cat|   0.66|                33.33|
+----------+-------+---------------------+


In [4]:
# solving in spark sql

queries_df.createOrReplaceTempView('queries')

spark.sql('''
select 
    query_name, 
    round(avg(rating/position),2) as quality,
    round(100*avg(case when rating<3 then 1 else 0 end),2) as poor_query_percentage
from queries
group by query_name;
''').show()

+----------+-------+---------------------+
|query_name|quality|poor_query_percentage|
+----------+-------+---------------------+
|       Dog|    2.5|                33.33|
|       Cat|   0.66|                33.33|
+----------+-------+---------------------+


In [5]:
spark.stop()